<a href="https://colab.research.google.com/github/Elizaluckianchikova/Biostatistics/blob/main/algbio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5.1 Глобальное выравнивание. Алгоритм Нидлмана-Вунша**


Глобальное выравнивание используется, когда мы хотим сравнить две последовательности от конца
до конца. Он используется, когда у нас есть последовательности примерно одинакового происхождения
и длины и хотим найти различия между ними. Например, оно может быть
например, для сравнения белков или геномов близкородственных организмов.
Прежде всего, используется идея простого взвешивания. Каждое совпадение стоит +1
балла, каждое несовпадение - µ баллов, каждый пробел - δ баллов. В результате получается

align_weight = #match + µ - #mismatch + δ - #gaps

Задача состоит в том, чтобы максимизировать общий вес выравнивания.
Задача может быть решена с помощью динамического программирования. Предположим, что нам известно оптимальное выравнивание последовательностей s[0 . . . i] и t[0 . . . j].
Тогда на текущем шаге мы можем выбрать сопряжение s[i+1] и t[j+1], чтобы получить совпадение или несовпадение,или мы можем выбрать выполнение разрыва. Повторяя эти действия, мы получаем все возможные выравнивания двух последовательностей.Для реализации этого алгоритма мы будем использовать таблицу W, а элемент W[i][j] будет хранить вес оптимального выравнивания последовательностей s[0 . . . i - 1] и t[0 . . . j - 1].
W[i][j] = max



W[i - 1][j - 1] + 1 s[i - 1] = t[j - 1], совпадение
W[i - 1][j - 1] + µ s[i - 1] ̸= t[j - 1], несовпадение
W[i][j - 1] + δ удаление
W[i - 1][j] + δ вставка
20
W[0][0] = 0, W[0][j] = -δ-j, W[i][0] = -δ-i.

Вес оптимального выравнивания
хранится в W[n][m].
Пример: µ = -1, δ = -2
G C A T G
0 -2 -4 -6 -8 -10
G -2 1 -1 -3 -5 -7
A -4 -1 0 0 -2 -4
T -6 -3 -2 -1 1 -1
T -8
T -8 -5 -4 -3 0 0
G-ATT
GCATG

In [ ]:
def calculate_W(s, t, match, mismatch, insertion, deletion):
    n = len(s)
    m = len(t)

    W = [[0] * (m+1) for _ in range(n+1)]

    for i in range(n+1):
        W[i][0] = i * deletion
    for j in range(m+1):
        W[0][j] = j * insertion

    for i in range(1, n+1):
        for j in range(1, m+1):
            if s[i-1] == t[j-1]:
                W[i][j] = max(W[i-1][j-1] + match, W[i-1][j] + deletion, W[i][j-1] + insertion)
            else:
                W[i][j] = max(W[i-1][j-1] + mismatch, W[i-1][j] + deletion, W[i][j-1] + insertion)

    return W

# Пример использования
s = "AGTACGCA"
t = "TATGC"
match = 1
mismatch = -1
insertion = -2
deletion = -2
result = calculate_W(s, t, match, mismatch, insertion, deletion)
for row in result:
    print(row)


**5.2 Глобальное выравнивание с помощью матрицы весов**
При сравнении двух аминокислотных последовательностей некоторые замены более вероятны чем другие, и могут привести к различным обстоятельствам. Поэтому лучше
использовать матрицу весов, а не присваивать каждому несовпадению одинаковый вес. На сайте
Две наиболее распространенные матрицы замен - PAM и BLOSUM.
- Матрица PAM (Point Accepted Mutation) строится с использованием информации о частоте единичных мутаций, основанной на наблюдениях за
последовательностей близкородственных белков. В матрице PAM_1 предполагается 1 мутация на
100 нуклеотидов. Часто используется матрица PAM_250.
- BLOSUM (BLOcks SUbstitution Matrix) строится на основе локальных
выравнивания последовательностей, сходство которых меньше заданного порога.
Часто используется матрица BLOSUM62, которая получается на основе последовательностей
со сходством менее 62 %.
Алгоритм меняется незначительно:
W[i][j] = max



W[i - 1][j - 1] + M[i-1][j-1] совпадение/несовпадение
W[i][j - 1] + δ удаление
W[i - 1][j] + δ вставка


In [ ]:
def calculate_W(s, t, M, insertion, deletion):
    n = len(s)
    m = len(t)

    W = [[0] * (m+1) for _ in range(n+1)]

    for i in range(n+1):
        W[i][0] = i * deletion
    for j in range(m+1):
        W[0][j] = j * insertion

    for i in range(1, n+1):
        for j in range(1, m+1):
            W[i][j] = max(W[i-1][j-1] + M[i-1][j-1], W[i-1][j] + deletion, W[i][j-1] + insertion)

    return W

# Example usage
s = "AGTACGCA"
t = "TATGC"
M = [[1, -1, 0, 0, 0],
     [-1, 1, 0, 0, 0],
     [0, 0, 1, -1, 0],
     [0, 0, -1, 1, 0],
     [0, 0, 0, 0, 1],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
insertion = -2
deletion = -2
result = calculate_W(s, t, M, insertion, deletion)
for row in result:
    print(row)


**5.3 Глобальное выравнивание с взвешенным штрафом за пробелы**


Каждый разрыв может быть связан с каким-то эволюционным событием, когда последовательность
была изменена. Последовательные пробелы могут быть представлены как события одной природы,
Таким образом, один мультигэп более вероятен, чем несколько одиночных гэпов.
A G C T T T T T T G C
A G C - T - T - G C менее вероятно
A G C - - - T T G C более вероятно
21
Чтобы алгоритм справился с этой ситуацией, нам нужна дополнительная
операция. Мы вводим штраф за открытие пробела, который обычно стоит много
по сравнению со штрафом за каждый индел.
gap_weight = ρ + i - σ
где ρ - штраф за открытие пробела, σ - штраф за расширение пробела, i - длина пробела.
Для эффективной реализации этой идеи используется "трехслойная манхэттенская сетка". Теперь
есть три матрицы, которые используются для оценки выравнивания:
- Главная - для диагональных ребер (совпадение/несовпадение)
- Нижний уровень - для горизонтальных ребер (удаление)
- Верхний уровень - для вертикальных ребер (вставка).
За переход с основного уровня на верхний или нижний назначается штраф за отсутствие пробелов.
верхний уровень или нижний уровень. Штраф за расширение зазора используется для каждого
шаг на нижний или верхний уровень.
upper[i][j] = max {
upper[i - 1][j] + σ продолжить вставку
main[i - 1][j] + ρ + σ начать вставку с main
lower[i][j] = max {
lower[i][j - 1] + σ продолжить удаление
main[i][j - 1] + ρ + σ начало удаления из main
main[i][j] = max



main[i - 1][j - 1] + M[i - 1][j - 1] совпадение/несовпадение
верхний[i][j] конец удаление
нижний[i][j] конец вставка

In [ ]:
def calculate_matrices(s, t, M, rho, sigma):
    n = len(s)
    m = len(t)

    upper = [[0] * (m+1) for _ in range(n+1)]
    lower = [[0] * (m+1) for _ in range(n+1)]
    main = [[0] * (m+1) for _ in range(n+1)]

    for i in range(1, n+1):
        upper[i][0] = float('-inf')
        lower[i][0] = float('-inf')
    for j in range(1, m+1):
        upper[0][j] = -rho - (j-1) * sigma
        lower[0][j] = float('-inf')

    for i in range(1, n+1):
        for j in range(1, m+1):
            upper[i][j] = max(upper[i-1][j] + sigma, main[i-1][j] + rho + sigma)
            lower[i][j] = max(lower[i][j-1] + sigma, main[i][j-1] + rho + sigma)
            main[i][j] = max(
                main[i-1][j-1] + M[i-1][j-1],
                upper[i][j],
                lower[i][j]
            )

    return upper, main, lower

# Example usage
s = "AGTACGCA"
t = "TATGC"
M = [[1, -1, 0, 0, 0],
     [-1, 1, 0, 0, 0],
     [0, 0, 1, -1, 0],
     [0, 0, -1, 1, 0],
     [0, 0, 0, 0, 1],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
rho = -2
sigma = -1
upper, main, lower = calculate_matrices(s, t, M, rho, sigma)
print("Upper Matrix:")
for row in upper:
    print(row)
print("\nMain Matrix:")
for row in main:
    print(row)
print("\nLower Matrix:")
for row in lower:
    print(row)


**5.4 Локальное выравнивание. Алгоритм Смита-Уотермана**

Локальное выравнивание используется для поиска наиболее схожей области между двумя последовательностями.
Например, можно найти один и тот же консервативный регион между двумя
различными последовательностями. При локальном выравнивании цель состоит в том, чтобы найти наилучшее выравнивание
между s[i . . . j] и t[i
′
. . . j′
].
Глобальное выравнивание:
A C T G A G
A G T C A -
22
Локальное выравнивание:
A C T G A G - - - - -
- - - - A G T C A
Наивный подход: вычислить глобальное выравнивание для всех возможных подпоследовательностей
s и t. Это займет O(n
4
), поскольку существует n
2 возможных начала.
Идея состоит в том, чтобы добавить 0-взвешенное ребро из (0, 0) в каждую ячейку (i, j). Это
означает, что мы будем считать вес для выравнивания, начинающегося только с этой
позиции. А чтобы найти наилучшее локальное выравнивание, нам нужно найти максимум
в матрице, поэтому остальные последовательности мы считать не будем. Это даст нам
23
желаемое O(n
2
) время.
W[i][j] = max



0 начало на [i, j]
W[i - 1][j - 1] + M[i - 1][j - 1] совпадение/несовпадение
W[i][j - 1] + δ удаление
W[i - 1][j] + δ вставка

In [ ]:
def calculate_W(s, t, M, insertion, deletion):
    n = len(s)
    m = len(t)

    W = [[0] * (m+1) for _ in range(n+1)]

    for i in range(1, n+1):
        for j in range(1, m+1):
            match_mismatch = W[i-1][j-1] + M[i-1][j-1]
            deletion_cost = W[i][j-1] + deletion
            insertion_cost = W[i-1][j] + insertion
            W[i][j] = max(0, match_mismatch, deletion_cost, insertion_cost)

    return W

# Example usage
s = "AGTACGCA"
t = "TATGC"
M = [[1, -1, 0, 0, 0],
     [-1, 1, 0, 0, 0],
     [0, 0, 1, -1, 0],
     [0, 0, -1, 1, 0],
     [0, 0, 0, 0, 1],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
insertion = -2
deletion = -2
result = calculate_W(s, t, M, insertion, deletion)
for row in result:
    print(row)


**5.5 Глобальное выравнивание с малым объемом памяти. Алгоритм Хиршберга**

Алгоритм Нидлмана-Вунша требует O(nm) памяти для хранения двумерной таблицы. Рассмотрим две последовательности длиной 5000 п.н. Для этого потребуется 5000 - 5000 - 8байт/сим =
∼ 200 МБ памяти. Это очень много.
Однако если нам нужна только стоимость оптимального выравнивания, то достаточно
хранить только две строки таблицы в каждый момент времени. С помощью рекурсивной формулы значения в
текущей строке можно вычислить, используя только предыдущую и текущую
строку. Таким образом, нам не нужно хранить матрицу. Результирующая память составит
O(min(n, m)).
Для восстановления самого выравнивания мы будем использовать принцип "разделяй и властвуй".
Для этого нам понадобятся два следующих свойства:
1. Глобальное выравнивание двух строк s и t равно обратному выравниванию
соответствующих перевернутых строк.
2. Если у нас (A, B) = Align(s, t) - результат выравнивания и s = sleft
+ sright, то существует разбиение t = tleft + tright, такое, что
Align(s, t) = Align(sleft, tleft) + Align(sright, tright)
Для описания алгоритма Хиршберга нам понадобится служебная функция Align_weight(s,
t), которая принимает на вход две строки и возвращает последнюю строку таблицы выравнивания
с весами. И помните, что она может быть реализована с линейной памятью,
храня только две строки за раз. Таким образом, алгоритм Хиршберга имеет вид:
d e f Hi r s c h b e r g ( s , t ) :
n = l e n ( s )
m = l e n ( t )
i f n == 0 :
r e t u r n '- ' ∗ m, t
i f m == 0 :
r e t u r n s , '- ' ∗ n
i f n == 1 или m == 1 :
r e t u r n Needleman_Wunsch ( s , t  )
r i g h t = Align_weight ( s [ xmid : ] [ : - 1 ] , t [ : - 1 ] ) [ : - 1 ]
ymid = argmax
i
( l e f t [ i ] + r i g h t [ i ] )
opt1 = Hi r s c h b e r g ( s [ : xmid ] , t [ : ymid ] )
opt2 = Hi r s c h b e r g ( s [ xmid : ] , t [ ymid : ] )
r e t u r n opt1 [ 0 ] + opt2 [ 0 ] , opt1 [ 1 ] + opt2 [ 1 ]
В базовом случае, когда последовательности достаточно малы (длина=1 в нашем случае)
используется классический алгоритм Нидлмана-Вунша. В противном случае мы вычисляем
веса оптимального выравнивания первой половины s с t, а второй
и второй половины s с t, используя обратный метод. Затем мы сравниваем результаты, чтобы
находим оптимальную точку разбиения для строки t. Наконец, мы рекурсивно повторяем
процедуру в левом верхнем и правом нижнем углах матрицы.
s = AGTACGCA, t = TATGC
Align_weight(AGTA, TATGC) = [-8, -4, 0, -2, -1, -3]
rev Align_weight(rev(CGCA), rev(TATGC)) = [-3, -1, 1, 0, -4, -8]
sum = [-11, -5, 1, -2, -5, -11]
Split = Hirschberg(AGTA, TA) + Hirschberg(CGCA, TGC)
s = AGTA, t = TA
Align_weight(AG, TA) = [-4, -3, -2]
rev Align_weight(rev(TA), rev(TA)) = [4, 0, -4]
sum = [0, -3, -6]
Split = Hirschberg(AG, ) + Hirschberg(TA, TA)
s = TA, t = TA
Align_weight(T, TA) = [-2, 2, 0]
rev Align_weight(rev(A), rev(TA)) = [0, 2, -2]
sum = [-2, 4, -2]
Split = Hirschberg(T, T) + Hirschberg(A, A)
25
s = CGCA, t = TGC
Align_weight(CG, TGC) = [-4, -3, 1, -1]
rev Align_weight(rev(CA), rev(TGC)) = [-4, -2, 0, -4]
sum = [-8, -5, 1, -5]
Split = Hirs
s = CGCA, t = TGC
Align_weight(CG, TGC) = [-4, -3, 1, -1]
rev Align_weight(rev(CA), rev(TGC)) = [-4, -2, 0, -4]
sum = [-8, -5, 1, -5]
Split = Hirschberg(CG, TG) + Hirschberg(CA, C)
s = CG, t = TG
Align_weight(C, TG) = [-2, -1, -3]
rev Align_weight(rev(G), rev(TG)) = [0, 2, -2]
sum = [-2, 1, -5]
Split = Hirschberg(C, T) + Hirschberg(G, G)

In [ ]:
def needleman_wunsch(s, t):
    n = len(s)
    m = len(t)
    dp = [[0] * (m+1) for _ in range(n+1)]
    for i in range(1, n+1):
        dp[i][0] = i
    for j in range(1, m+1):
        dp[0][j] = j
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = dp[i-1][j-1] + (1 if s[i-1] == t[j-1] else -1)
            delete_s = dp[i-1][j] + 1
            delete_t = dp[i][j-1] + 1
            dp[i][j] = min(match, delete_s, delete_t)
    return dp[n][m]

def align_weight(s, t):
    n = len(s)
    m = len(t)
    prev = list(range(m+1))
    for i in range(1, n+1):
        curr = [i] + [0] * m
        for j in range(1, m+1):
            match = prev[j-1] + (1 if s[i-1] == t[j-1] else -1)
            delete_s = prev[j] + 1
            delete_t = curr[j-1] + 1
            curr[j] = min(match, delete_s, delete_t)
        prev = curr
    return prev

def hirschberg(s, t):
    n = len(s)
    m = len(t)
    if n == 0:
        return '-' * m, t
    if m == 0:
        return s, '-' * n
    if n == 1 or m == 1:
        return needleman_wunsch(s, t)

    xmid = n // 2
    left = align_weight(s[:xmid], t)
    right = align_weight(s[xmid:][::-1], t[::-1])[::-1]
    ymid = max(range(m+1), key=lambda i: left[i] + right[i])

    opt1 = hirschberg(s[:xmid], t[:ymid])
    opt2 = hirschberg(s[xmid:], t[ymid:])

    return opt1[0] + opt2[0], opt1[1] + opt2[1]

# Пример использования
s = "AGTACGCA"
t = "TATGC"
result = hirschberg(s, t)
print(result)


**6.1 Вторичные структуры**

Последовательность РНК строится из нуклеотидов из набора {A, U, G, C}. Как правило, РНК
одноцепочечная, но нуклеотиды могут образовывать комплементарные связи, поэтому она
складывается во вторичную структуру.
Комплементарность оснований: A = U, G ≡ C.
Формально набор всех комплементарных пар последовательности s можно записать как
P(S) ⊂ {(i, j) | 1 ≤ i < j ≤ n, comp(s[i], s[j])}.
Может быть две различные структуры: вложенная и пересекающаяся.
Структура является вложенной, если любые две комплементарные пары не пересекаются. В противном случае структура называется пересекающейся.
Пример:
6.1.1 Способы представления вторичной структуры РНК
- Круговая схема - последовательность записывается в виде круга, а комплементарные
основания соединены хордами
- Скобочная схема - последовательность записывается с помощью символов точки и скобок
1. Если нуклеотид ci непарный, в позиции i выводится .
2. если нуклеотиды ci и cj парные, выведите ( в позиции i и ) в позиции j
в позиции j
GCUGAUGGCAU
27
. ( . ) ( ( ( . ) ) )
[0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 4]
[0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def bracket_representation(sequence, pairs):
    s = ['.'] * len(sequence)
    for i, j in pairs:
        s[i] = '('
        s[j] = ')'
    return ''.join(s)

sequence = "GCUGAUGGCAU"
pairs = [(3, 10), (4, 9), (5, 8), (6, 7)]
bracket_seq = bracket_representation(sequence, pairs)
print(sequence)
print(bracket_seq)


**Алгоритм Нуссинова**

N[i][j] - размер максимальной вложенной структуры для последовательности s[i . . . j]. N[i][i] = 0,
N[i][j] = 0 ∀i > j. Предположим, что мы хотим найти оптимальную структуру для последовательности
s[i . . . j] и знаем оптимальные структуры для всех меньших подпоследовательностей. Тогда
существует четыре случая:
1. Возьмем элементы в позициях i и j, объединим их в пары, если они дополняют друг друга,
добавляем их в структуру s[i + 1 ... j - 1].
2. Добавить элемент в позиции i в структуру s[i + 1 . . . j]
3. Добавьте элемент в позиции j в структуру s[i . . . j - 1]
4. Выберите k между i и j и присоедините к оптимальным последовательностям s[i . . . k] и s[k +
1 . . . j]
N[i][j] = max



N[i + 1][j - 1] + wi,j
max
i≤k<j
N[i][k] + N[k + 1][j]
Ответ находится в точке N[0][n - 1], поскольку мы хотим найти оптимальную структуру
для всей последовательности.
28
Чтобы найти оптимальную структуру, нам нужно проследить по матрице
и заполнить соответствующие значения строки ответа.

In [ ]:
def calculate_N(weights):
    n = len(weights)
    N = [[0] * n for _ in range(n)]

    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            N[i][j] = max(N[i+1][j-1] + weights[i][j], max(N[i][k] + N[k+1][j] for k in range(i, j)))

    return N

# Пример использования
weights = [
    [0, 2, 3, 4],
    [0, 0, 1, 2],
    [0, 0, 0, 3],
    [0, 0, 0, 0]
]
result = calculate_N(weights)
for row in result:
    print(row)
